In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from lifelines import CoxPHFitter
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA

In [2]:
npz_data = np.load('fusion_features_origin.npz')
features = npz_data['features']  # Shape (144, 1024)
patient_ids = npz_data['patient_ids']  # Shape (144,)

event_data = pd.read_csv('processed_data.csv')

In [3]:
feature_columns = [f'feature_{i}' for i in range(features.shape[1])]
features_df = pd.DataFrame(features, columns=feature_columns)

features_df['Case ID'] = patient_ids
features_df.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_1015,feature_1016,feature_1017,feature_1018,feature_1019,feature_1020,feature_1021,feature_1022,feature_1023,Case ID
0,0.0,0.0,0.0,0.389201,0.0,0.0,0.0,1.410983,0.000000,0.066649,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.002345,0.000000,0.0,lung_001
1,0.0,0.0,0.0,0.247796,0.0,0.0,0.0,1.444206,0.432007,0.359523,...,0.000000,0.0,0.082699,0.0,0.084822,0.000000,0.000000,0.000000,0.0,lung_002
2,0.0,0.0,0.0,0.296023,0.0,0.0,0.0,1.438157,0.000000,0.000000,...,0.471473,0.0,0.218094,0.0,0.000000,0.000000,0.000000,1.525748,0.0,lung_003
3,0.0,0.0,0.0,0.504906,0.0,0.0,0.0,1.296884,0.848339,0.000000,...,0.596818,0.0,0.048936,0.0,0.044266,0.000000,0.084124,1.500630,0.0,lung_004
4,0.0,0.0,0.0,0.285188,0.0,0.0,0.0,1.161725,0.425602,0.006042,...,0.507732,0.0,0.000000,0.0,0.000000,0.007567,0.000000,0.000000,0.0,lung_005


In [4]:
event_data.head()

,Case ID,Age,Weight (lbs),Gender,Ethnicity,Smoking status,%GG,Tumor Location (choice=RUL),Tumor Location (choice=RML),Tumor Location (choice=RLL),...,Std,Min,Max,Median,SurfaceArea,Elongation,Flatness,Roundness,Time to Event,Event
0,lung_001,79,146.0,0,2,1,0,0,1,1,...,194.164635,-812,154,-56.236328,826.137989,1.436361,1.446431,0.699813,3078,0
1,lung_002,65,195.0,0,1,2,0,0,1,1,...,173.439744,-829,144,-36.021484,1037.374063,1.367921,1.112439,0.781205,70,0
2,lung_003,65,173.5,1,2,0,0,1,1,1,...,172.357348,-815,290,23.177734,755.268235,1.387373,1.089084,0.822009,666,0
3,lung_004,67,173.5,1,2,1,0,1,1,1,...,254.147443,-1024,366,21.595703,912.514223,1.593605,1.631120,0.639694,1172,0
4,lung_005,84,145.0,1,4,1,0,1,0,1,...,107.583454,-783,391,7.496094,2432.305090,1.278080,1.166290,0.735654,1456,1


In [5]:
event_data_subset = event_data[['Case ID', 'Time to Event', 'Event']]
combined_df = pd.merge(features_df, event_data_subset, on='Case ID', how='inner')
combined_df.shape[0]

144

In [6]:
features_cols = [col for col in combined_df.columns if col.startswith('feature_')]
metadata_cols = [col for col in combined_df.columns if not col.startswith('feature_')]

X = combined_df[features_cols]
metadata = combined_df[metadata_cols]

In [7]:
n_components = 50  # Bạn có thể thay đổi giá trị này

# Áp dụng PCA để giảm chiều
pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X)

In [8]:
explained_variance = np.sum(pca.explained_variance_ratio_)
print(f"Số chiều ban đầu: {X.shape[1]}")
print(f"Số chiều sau PCA: {n_components}")
print(f"Tỷ lệ phương sai giải thích được: {explained_variance:.4f} ({explained_variance*100:.2f}%)")

Số chiều ban đầu: 1024
Số chiều sau PCA: 50
Tỷ lệ phương sai giải thích được: 0.7027 (70.27%)


In [9]:
pca_cols = [f'pca_{i}' for i in range(X_pca.shape[1])]
X_pca_df = pd.DataFrame(X_pca, columns=pca_cols)
pca_df = pd.concat([X_pca_df, metadata], axis=1)

In [10]:
train_df, test_df = train_test_split(pca_df, test_size=0.2, random_state=42)

In [11]:
train_df = train_df.drop(columns=['Case ID'])
test_df = test_df.drop(columns=['Case ID'])

In [12]:
cph = CoxPHFitter(penalizer=0.1, l1_ratio=0.5)
cph.fit(train_df, duration_col='Time to Event', event_col='Event')

<lifelines.CoxPHFitter: fitted with 115 total observations, 73 right-censored observations>

In [13]:
cph.print_summary()  # access the individual results using cph.summary
with open("results_CoxPH_PCA_origin.txt", "w") as f:
    f.write(cph.summary.to_string() + "\n")
    f.write(f"Concordance Index: {cph.concordance_index_}\n")

<lifelines.CoxPHFitter: fitted with 115 total observations, 73 right-censored observations>
             duration col = 'Time to Event'
                event col = 'Event'
                penalizer = 0.1
                 l1 ratio = 0.5
      baseline estimation = breslow
   number of observations = 115
number of events observed = 42
   partial log-likelihood = -180.38
         time fit was run = 2025-04-26 16:46:06 UTC

---
           coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                  
pca_0      0.17      1.18      0.14           -0.10            0.43                0.91                1.54
pca_1      0.00      1.00      0.00           -0.00            0.00                1.00                1.00
pca_2      0.02      1.02      0.14           -0.26            0.30                0.77                1.35
pca_3     -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
pca_4     -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
pca_5     -0.03      0.97      0.17           -0.36            0.30                0.70                1.35
pca_6      0.00      1.00      0.00           -0.00            0.00                1.00                1.00
pca_7      0.02      1.02      0.16           -0.29            0.34                0.75                1.40
pca_8      0.00      1.00      0.00           -0.00            0.00                1.00                1.00
pca_9     -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
pca_10    -0.02      0.98      0.19           -0.40            0.36                0.67                1.43
pca_11     0.03      1.03      0.17           -0.30            0.35                0.74                1.42
pca_12     0.00      1.00      0.00           -0.00            0.00                1.00                1.00
pca_13    -0.04      0.96      0.19           -0.40            0.33                0.67                1.39
pca_14     0.00      1.00      0.00           -0.00            0.00                1.00                1.00
pca_15     0.00      1.00      0.00           -0.00            0.00                1.00                1.00
pca_16    -0.02      0.98      0.18           -0.37            0.34                0.69                1.40
pca_17    -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
pca_18     0.00      1.00      0.00           -0.00            0.00                1.00                1.00
pca_19    -0.06      0.94      0.20           -0.45            0.32                0.64                1.38
pca_20     0.00      1.00      0.00           -0.00            0.00                1.00                1.00
pca_21    -0.03      0.97      0.20           -0.43            0.37                0.65                1.45
pca_22    -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
pca_23    -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
pca_24    -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
pca_25     0.00      1.00      0.00           -0.00            0.00                1.00                1.00
pca_26     0.12      1.13      0.21           -0.28            0.53                0.76                1.69
pca_27    -0.13      0.88      0.23           -0.59            0.33                0.56                1.39
pca_28    -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
pca_29     0.00      1.00      0.00           -0.00            0.00                1.00                1.00
pca_30     0.00      1.00      0.00           -0.00            0.00                1.00                1.00
pca_31  

In [14]:
test_ci = cph.score(test_df, scoring_method="concordance_index")
print(test_ci)

0.4540540540540541
